In [1]:
import datetime
import time
from datetime import datetime, date, timedelta
import pandas as pd
import numpy as np
from pathlib import Path
from pandas.testing import assert_frame_equal
import matplotlib.pyplot as plt
from matplotlib.cm import Greys
import matplotlib.dates as mdt
import seaborn as sns
import scipy.stats as ss
p1 = Path.cwd() / 'back_data'

In [2]:
from zipfile import ZipFile

In [6]:
# zipfile.ZipFile() 함수 활용하여 압축파일 열기 진행
with ZipFile(p1 / 'kaggle-survey-2018.zip') as file:
    # pandas의 read_csv() 함수에 압축파일 내 파일을 open() 메서드 활용하여 전달
    kag = pd.read_csv(file.open('multipleChoiceResponses.csv'))
    df = kag.iloc[1:]
df.head()

C:\Users\admin\AppData\Local\Temp\ipykernel_4392\575755967.py:4: DtypeWarning: Columns (0,2,8,10,21,23,24,25,26,27,28,44,56,64,83,85,87,107,109,123,125,150,157,172,174,194,210,218,219,223,246,249,262,264,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,304,306,325,326,329,341,368,371,384,385,389,390,391,393,394) have mixed types. Specify dtype option on import or set low_memory=False.
  kag = pd.read_csv(file.open('multipleChoiceResponses.csv'))


,Time from Start to Finish (seconds),Q1,Q1_OTHER_TEXT,Q2,Q3,Q4,Q5,Q6,Q6_OTHER_TEXT,Q7,...,Q49_OTHER_TEXT,Q50_Part_1,Q50_Part_2,Q50_Part_3,Q50_Part_4,Q50_Part_5,Q50_Part_6,Q50_Part_7,Q50_Part_8,Q50_OTHER_TEXT
1,710,Female,-1,45-49,United States of America,Doctoral degree,Other,Consultant,-1,Other,...,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
2,434,Male,-1,30-34,Indonesia,Bachelor’s degree,Engineering (non-computer focused),Other,0,Manufacturing/Fabrication,...,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
3,718,Female,-1,30-34,United States of America,Master’s degree,"Computer science (software engineering, etc.)",Data Scientist,-1,I am a student,...,-1,NaN,Too time-consuming,NaN,NaN,NaN,NaN,NaN,NaN,-1
4,621,Male,-1,35-39,United States of America,Master’s degree,"Social sciences (anthropology, psychology, soc...",Not employed,-1,NaN,...,-1,NaN,NaN,Requires too much technical knowledge,NaN,Not enough incentives to share my work,NaN,NaN,NaN,-1
5,731,Male,-1,22-24,India,Master’s degree,Mathematics or statistics,Data Analyst,-1,I am a student,...,-1,NaN,Too time-consuming,NaN,NaN,Not enough incentives to share my work,NaN,NaN,NaN,-1


In [12]:
# 1번 문항의 구성 및 도수 확인
df['Q1'].value_counts()
q1 = (df['Q1']
# 키를 튜플을 활용하는 것은 불가능 -> 하나씩 바꿔줘야
.replace({'Prefer not to say':'Another', 'Prefer to self-describe':'Another'})
.rename('Gender')
)
q1.value_counts()

Male       19430
Female      4010
Another      419
Name: Gender, dtype: int64

In [13]:
# 2번 문항의 구성 및 도수 확인
df['Q2'].value_counts()
q2 = (df['Q2']
# str 속성의 slice() 메서드를 활용해 앞 두자리만 추출하고 정수형으로 변환
.str.slice(0, 2)
.astype('int')
.rename('Age')
)
q2.value_counts()

25    6159
22    5141
30    3776
18    3037
35    2253
40    1360
45     858
50     582
55     328
60     273
70      53
80      39
Name: Age, dtype: int64

In [53]:
# 3번 문항의 구성 및 도수 확인
df['Q3'].value_counts()
q3 = (df['Q3']
# where() 메서드 활용 -> value_counts()와 index 속성 연결하여 상위 n개 나라 추출
.where(df['Q3'].isin(df['Q3'].value_counts().index[:8]), 'Other')
.replace('United Kingdom of Great Britain and Northern Ireland', 'Britain')
.fillna('Not response')
.rename('Country')
)
q3.value_counts()

Other                       10031
United States of America     4716
India                        4417
China                        1644
Russia                        879
Brazil                        736
Germany                       734
Britain                       702
Name: Country, dtype: int64

In [21]:
# 4번 문항의 구성 및 도수 확인
df['Q4'].value_counts()
# 각 degree를 숫자로 점수화 하여 변환
q4 = (df['Q4']
.replace({'Master’s degree': 18, 'Bachelor’s degree': 16, 'Doctoral degree': 20,
'Some college/university study without earning a bachelor’s degree': 13, 'Professional degree': 19,
# 책에는 None이라고 써져 있으나 pandas 내용이므로 np.nan이 더 적절할듯
'I prefer not to answer': np.nan, 'No formal education past high school': 12})
# 항목이 없을 경우 11점으로 통일
.fillna(11)
.rename('Edu')
)
q4.value_counts()

18.0    10855
16.0     7083
20.0     3357
13.0      967
11.0      766
19.0      599
12.0      232
Name: Edu, dtype: int64

In [34]:
# 5번 문항의 구성 및 도수 확인
df['Q5'].value_counts()
q5 = (df['Q5']
# where() 메서드 활용 -> value_counts()와 index 속성 연결하여 상위 n개 나라 추출
.where(df['Q5'].isin(df['Q5'].value_counts().index[:3]), 'Other')
# replace() 메서드 활용하여 긴 이름들을 약어로 전환
.replace({'Computer science (software engineering, etc.)': 'cs', 'Engineering (non-computer focused)': 'eng',
'Mathematics or statistics': 'stat'})
.fillna('Not response')
.rename('Studies')
)
q5.value_counts()

cs       9430
Other    7774
eng      3705
stat     2950
Name: Studies, dtype: int64

In [33]:
# 6번 문항의 구성 및 도수 확인
df['Q6'].value_counts()
q6 = (df['Q6']
# where() 메서드 활용 -> df['Q6'].isin() 메서드에 포함될 리스트 그대로 전달
.where(df['Q6'].isin(['Student', 'Data Scientist', 'Software Engineer', 'Not employed', 'Data Engineer']), 'Other')
.fillna('Not response')
.rename('Occupation')
)
q6.value_counts()

Other                9760
Student              5253
Data Scientist       4137
Software Engineer    3130
Not employed          842
Data Engineer         737
Name: Occupation, dtype: int64

In [32]:
# 8번 문항의 구성 및 도수 확인
df['Q8'].value_counts()
q8 = (df['Q8']
# str 속성에서 앞 두자리를 추출한 뒤 '-'를 빈 값으로 변환
.str.slice(0, 2)
.str.replace('-', '')
.fillna(0)
.astype('int')
.rename('Experience')
)
q8.value_counts()

0     8656
1     3745
2     2577
5     2524
3     1751
10    1512
4     1488
15     854
20     384
30     197
25     171
Name: Experience, dtype: int64

In [51]:
# 9번 문항의 구성 및 도수 확인
df['Q9'].value_counts()
q9 = (df['Q9']
# str 속성의 replace() 메서드는 딕셔너리 활용 불가능인듯
.str.replace('+', '')
.str.replace(',000', '')
.str.strip()
# str 속성의 split() 메서드 이용하여 '-'를 기준으로 분할하고 expand=True 인자 전달한 후 첫 번째 열만 수집
.str.split('-', expand=True).iloc[:, 0]
# 미응답 답변들의 경우 일단 일정한 숫자로 변환
.replace('I do not wish to disclose my approximate yearly compensation', 1000)
.fillna(1000)
.astype('int')
.mul(1000)
# 미응답 답변을 'Not Response'로 변환
.replace(1000000, 'Not response')
.rename('Salary')
)
q9.value_counts()

C:\Users\admin\AppData\Local\Temp\ipykernel_4392\2735867446.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  q9 = (df['Q9']


Not response    8430
0               4398
10000           1937
20000           1395
30000           1119
40000            965
50000            919
100000           843
60000            729
70000            677
90000            566
125000           533
80000            506
150000           457
200000           172
250000            75
500000            63
300000            52
400000            23
Name: Salary, dtype: int64

In [54]:
# 생성한 문항들을 열을 기준으로 pandas concat() 함수 이용하여 병합
pd.concat([q1, q2, q3, q4, q5, q6, q8, q9], axis=1)

,Gender,Age,Country,Edu,Studies,Occupation,Experience,Salary
1,Female,45,United States of America,20.0,Other,Other,0,Not response
2,Male,30,Other,16.0,eng,Other,5,10000
3,Female,30,United States of America,18.0,cs,Data Scientist,0,0
4,Male,35,United States of America,18.0,Other,Not employed,0,Not response
5,Male,22,India,18.0,stat,Other,0,0
...,...,...,...,...,...,...,...,...
23855,Male,45,Other,20.0,cs,Other,5,250000
23856,Female,25,Other,18.0,eng,Other,0,Not response
23857,Male,22,Other,18.0,cs,Software Engineer,0,10000
23858,Male,25,Britain,11.0,Other,Other,0,Not response
